## Read Khan Academy Lessons Mongo Collection and generate Word Templates

In [1]:
import os
import pymongo
import pandas as pd
import re
import docx
from docx.shared import Pt
from docx.shared import RGBColor
from docx.oxml.ns import qn

In [9]:
# Script Variables

# subject = 'High school statistics'
subject = 'Statistics and probability'
# subject = 'AP®︎/College Statistics'

# Target directory
root_dir = 'd:\\Primeway\\Treinamento\\Data Science & AI\\Statistics'



In [16]:
# Connect to MongoDB and load lessons collection into Pandas Dataframe

client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['khan-academy']

query = { 'subject': subject, 'lesson_type': { "$in": [ "Article", "Video" ] } }
columns = {'_id': 0, 'subject': 0, 'unit_sequence': 0, 'lesson_type': 0, 'lesson_link': 0 }

lessons = db.lessons.find(query, columns)

list_lessons = list(lessons)
df = pd.DataFrame(list_lessons)

df.tail()

,unit,group,lesson
393,Unit 15: Advanced regression (inference and tr...,Nonlinear regression,Transforming nonlinear data
394,Unit 15: Advanced regression (inference and tr...,Nonlinear regression,Worked example of linear regression using tran...
395,Unit 16: Analysis of variance (ANOVA),Analysis of variance (ANOVA),ANOVA 1: Calculating SST (total sum of squares)
396,Unit 16: Analysis of variance (ANOVA),Analysis of variance (ANOVA),ANOVA 2: Calculating SSW and SSB (total sum of...
397,Unit 16: Analysis of variance (ANOVA),Analysis of variance (ANOVA),ANOVA 3: Hypothesis test with F-statistic


In [31]:
df.index
print(df.size)

1194


In [27]:
# Helper Functions

# Prepare Word Document
def prepare_document(unit):

   # https://python-docx.readthedocs.io/en/latest/user/styles-using.html

   document = docx.Document()

   # Styling the Document

   # Normal
   style_normal = document.styles['Normal']
   style_normal.font.name = 'Calibri'
   style_normal.font.size = Pt(11)

   # Title
   style_title = document.styles['Title']
   style_title.font.name = 'Calibri Light'
   style_title.font.size = Pt(28)
   style_title.font.color.rgb = RGBColor(0x00, 0x00, 0x00)
   # https://stackoverflow.com/questions/60921603/how-do-i-change-heading-font-face-and-size-in-python-docx
   rFonts = style_title.element.rPr.rFonts
   rFonts.set(qn("w:asciiTheme"), "Calibri Light")

   # Heading 1
   style_heading1 = document.styles['Heading 1']
   style_heading1.font.name = 'Calibri Light'
   style_heading1.font.bold = None
   style_heading1.font.size = Pt(16)
   style_heading1.font.color.rgb = RGBColor(0x2F, 0x54, 0x96)
   rFonts = style_heading1.element.rPr.rFonts
   rFonts.set(qn("w:asciiTheme"), "Calibri Light")

   # Heading 2
   style_heading2 = document.styles['Heading 2']
   style_heading2.font.name = 'Calibri Light'
   style_heading2.font.bold = None
   style_heading2.font.size = Pt(13)
   style_heading2.font.color.rgb = RGBColor(0x2F, 0x54, 0x96)
   rFonts = style_heading2.element.rPr.rFonts
   rFonts.set(qn("w:asciiTheme"), "Calibri Light")

   # Generate the document content

   document.add_heading(unit, 0)

   return document

# Sanitize filename and save document
def save_document(document, unit):
   filename = re.sub(r':'," -", unit)
   filename = re.sub(r'[\\/*?"<>|]',"", filename)
   filename = filename + '.docx'

   # Check if the file exists
   path = os.path.join(os.getcwd(), filename)
   # print(path)
   if not os.path.isfile(path):
      document.save(filename)

   

In [12]:
# Create and/or move to target directory

os.chdir(root_dir)
parent_dir = os.getcwd()

path = os.path.join(parent_dir, subject)
print(path)

# Create the directory
if not os.path.isdir(path):
   os.mkdir(path)

os.chdir(path)
os.getcwd()



d:\Primeway\Treinamento\Data Science & AI\Statistics\Statistics and probability


'd:\\Primeway\\Treinamento\\Data Science & AI\\Statistics\\Statistics and probability'

In [33]:
# Interact through Dataframe and save doc file

df_length = len(df.index)

for i in df.index:

    # Nem unit
    if i == 0 or df['unit'][i] != unit:          
        unit = df['unit'][i]
        document = prepare_document(re.sub(r':'," -", unit))
        group = ''

    # New Group
    if df['group'][i] != group: 
        if group != '':
            document.add_page_break()
        document.add_heading(df['group'][i], level=1)
        document.add_paragraph('', style='Normal') 
        group = df['group'][i]

    document.add_heading("\t" + df['lesson'][i], level=2)
    document.add_paragraph('\t', style='Normal')
    document.add_paragraph('\t', style='Normal')
    document.add_paragraph('\t', style='Normal')

    if (i == df_length-1) or (df['unit'][i+1] != unit):
        save_document(document, unit)
